In [1]:
!pip install spacy 
!pip install neuralcoref
!pip install flair
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/Kode/NLP_project.zip -d /content
!cp /content/code/sentimentAnalysis.py /content

!cp /content/code/preprocessing_util/utils.py /content

import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')
!python -m spacy download en_core_web_sm

import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')
import re
import string
from itertools import combinations
from collections import Counter


import itertools
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from flair.models import SequenceTagger
from flair.data import Sentence
import operator
import json
import os

     |████████████████████████████████| 286 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 55.1 MB/s 
     |████████████████████████████████| 79 kB 2.1 MB/s 
     |████████████████████████████████| 8.8 MB 69.0 MB/s 
     |████████████████████████████████| 127 kB 69.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 401 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 63.6 MB/s 
     |████████████████████████████████| 981 kB 59.5 MB/s 
     |████████████████████████████████| 1.6 MB 38.1 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 


NameError: ignored

In [2]:

def read_text(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
        text = text.replace('\r', ' ').replace('\n', ' ')\
            .replace("’", "'").replace("\"", "").replace("”", "").replace("“", "")
    return text


def flair_NER(book):
    """
    flair_NER vrne seznam, v katerem so shranjene prepoznane identitete glede na posamezni stavek
    :param book: str
    :return: entity_dict (seznam slovarjev kot npr. [name, tag, start_pos, stop_pos, line_num. token_num])
    """

    # 00 Pretvori knjigo v stavke:
    sent = sent_tokenize(book)

    # 01 NER model
    entity_dict = []
    for line_num, line in enumerate(tqdm(sent)):

        # 01a Predict:
        sentence = Sentence(line)
        tagger.predict(sentence)

        # 01b Vmesni izpis (brez lokacije):
        #print(sentence.to_tagged_string())
        #print(sentence.get_spans('ner'))

        # 01c Ekstrakcija podatka iz stavka:
        for entity in sentence.get_spans('ner'):        # veliko različnih flair - nerov
            name = entity.text

            # str location
            # start_pos = entity.start_position    # št. str
            # stop_pos = entity.end_position      # št. str

            # token location
            tmp_flag = True
            for token in entity:
                if tmp_flag:
                    start_pos = token.idx -1
                    stop_pos = token.idx
                else:
                    stop_pos = token.idx
                tmp_flag = False

            tag = entity.get_label("ner").value             # tag = entity.tag
            conf_score = entity.get_label("ner").score      # conf_score = entity.score

            info_dict = {}
            info_dict["name"] = name
            info_dict["tag"] = tag
            info_dict["start_pos"] = start_pos
            info_dict["stop_pos"] = stop_pos
            info_dict["line_num"] = line_num

            entity_dict.append(info_dict)

    return entity_dict


def get_names_from_NER(entity_dict):
    """
    get_names_from_NER sprejme entity_dict in vrne urejen seznam terk ("ime", št_zaznano)
    :param entity_dict: dict (seznam dictov)
    :return: unique_names: list
    """
    unique_names = {}

    for entity in entity_dict:
        if entity["tag"] == "PER":
            if entity["name"] not in unique_names:
                unique_names[entity["name"]] = 1
            else:
                unique_names[entity["name"]] += 1
    unique_names = sorted(unique_names.items(), key=operator.itemgetter(1), reverse=True)

    return unique_names


def safe_open_w(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    os.makedirs(os.path.dirname(path), exist_ok=True)
    return open(path, 'w')

In [ ]:
import spacy
import neuralcoref

from utils import readWholeBook, readBookByChapters, getChaptersInOrder
nlp = spacy.load('en_core_web_sm')  # load the model
neuralcoref.add_to_pipe(nlp)

data_path = "/content/data/books/ASongOfIceAndFire/AGOT/chapters/"
results_path = "/content/results/books/ASongOfIceAndFire/AGOT/chapters"
chapters = getChaptersInOrder(data_path)

for chapter in chapters:
    text = read_text(data_path + chapter)
    doc = nlp(text)
    entitiy_dict = flair_NER(doc._.coref_resolved)
    unique_names = get_names_from_NER(entitiy_dict)

    chapter = chapter.replace(".txt", "")

    with safe_open_w("/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref/" + chapter + "_entity_dict") as f:
        json.dump(entitiy_dict, f)

    with safe_open_w("/content/results/books/ASongOfIceAndFire/AGOT/chapters_coref/" + chapter + "_unique_names") as f:
        json.dump(unique_names, f)

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:01<00:00, 36213991.72B/s]
